# QLoRA Tuning on Llama2

We will use the QLoRA technique to fine-tune the model in 4-bit precision and optimize VRAM usage.

Techniques applying:
- Quantization: HuggingFace Transformers has integrated optimum API to perform GPTQ quantization on LM. We can load and quantize our models in 8,4,3 or even 2 bits without a big drop of performance and still achieve faster inference speeds. This is achieved with the `BitsAndBytesConfig`. 
- LoRA: Stands for Low-rank Adaptation. It's widely used and effective for training custom LLMs. Read the paper [here](https://arxiv.org/abs/2305.14314).
- When you put quantization and LoRA together, we get QLoRA. Which, theoretically, reduces memory usage well.

In [1]:
import os
import pandas as pd
import creds

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer


/home/bobby/code-repo/practices/llm-examples/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Prepare some variables
## model from HF hub
base_model_name = 'meta-llama/Llama-2-7b-hf'

## New Insturctional Dataset
instructional_dataset = 'mlabonne/guanaco-llama2-1k'

## Folder name to store finetuned model
folder_name = 'meta-llama2-7b-chat-guanaco'

### 1. Load the Dataset

In [4]:
dataset = load_dataset(instructional_dataset, split='train')

In [5]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [6]:
## view the dataset
print(pd.DataFrame(dataset).iloc[0]['text'])

<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>


In [7]:
print(len(dataset))

## This is good. Optimally, we want the instructional prompts to be ~1000.

1000


### 2. Prepare 4-bit quantization configuration

In [8]:
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True, # data will be loaded in 4-bit format
    bnb_4bit_quant_type='nf4', # a quantizsation type
    bnb_4bit_compute_dtype=compute_dtype, # torch's float16
    bnb_4bit_use_double_quant=False # double quantization will not be used
)

### 3. Load Llama2 Model

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config, 
    device_map='auto', # automatically sets the device mapping
    # token = creds.HUGGINGFACE_TOKEN
)

model.config.use_cache = False # disables the use of cache in the model config
model.config.pretraining_tp = 1 # sets the pretraining temperature parameter to 1 in the model config

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.51s/it]


### 4. Load Llama2 Tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name,trust_remote_code=True)#,token=creds.HUGGINGFACE_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### 5. Preparing the PEFT Parameters

In [11]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

### 6. Training Parameters

In [12]:
training_params = TrainingArguments(
    output_dir="./results/meta-llama2-7b",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### 7. Model Fine-tuning

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text", # specifies the field in the dataset that contains text to be processed
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/bobby/code-repo/practices/llm-examples/env/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [14]:
## train the model
trainer.train()

Step,Training Loss
25,1.214800
50,1.527100
75,1.188800
100,1.390700
125,1.157700
150,1.339200
175,1.154800
200,1.427900
225,1.128200
250,1.471600


TrainOutput(global_step=250, training_loss=1.300079948425293, metrics={'train_runtime': 1075.3707, 'train_samples_per_second': 0.93, 'train_steps_per_second': 0.232, 'total_flos': 1.7122838399778816e+16, 'train_loss': 1.300079948425293, 'epoch': 1.0})

In [31]:
## train the model
trainer.train()

Step,Training Loss
25,1.215100
50,1.530300
75,1.188700
100,1.391100
125,1.158300
150,1.339200
175,1.154000
200,1.427600
225,1.126900
250,1.472000


TrainOutput(global_step=250, training_loss=1.3003206558227538, metrics={'train_runtime': 1076.9885, 'train_samples_per_second': 0.929, 'train_steps_per_second': 0.232, 'total_flos': 1.7122838399778816e+16, 'train_loss': 1.3003206558227538, 'epoch': 1.0})

In [15]:
## save the model and tokenizer
trainer.model.save_pretrained(folder_name)#, token = creds.HUGGINGFACE_TOKEN)
trainer.tokenizer.save_pretrained(folder_name)

## be sure to login to the HF CLI first so that you can save the pretrained. ALternatively, you can use token = creds.HUGGINGFACE_TOKEN

('meta-llama2-7b-chat-guanaco/tokenizer_config.json',
 'meta-llama2-7b-chat-guanaco/special_tokens_map.json',
 'meta-llama2-7b-chat-guanaco/tokenizer.json')

In [16]:
## load the model you had saved
loaded_model = AutoModelForCausalLM.from_pretrained(f"./{folder_name}",
                                                    quantization_config = quant_config,
                                                    device_map='auto')
loaded_model.config.use_cache = False
loaded_model.config.pretraining_tp = 1

loaded_tokenizer = AutoTokenizer.from_pretrained(f"./{folder_name}", trust_remote_code=True)
loaded_tokenizer.pad_token = loaded_tokenizer.eos_token
loaded_tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


In [34]:
from tensorboard import notebook

log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

Reusing TensorBoard on port 4000 (pid 2840145), started 21:15:05 ago. (Use '!kill 2840145' to kill it.)

### 8. Inferencing

In [17]:
## Inference with SAVED MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe_loaded = pipeline(task="text-generation", model=loaded_model, tokenizer=loaded_tokenizer, max_length=200)
result_loaded = pipe_loaded(f"<s>[INST] {prompt} [/INST]")
print(result_loaded[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci was an Italian Renaissance polymath: painter, sculptor, architect, musician, mathematician, engineer, inventor, anatomist, geologist, cartographer, botanist, and writer. 150 years before the invention of the telescope and 100 years before the invention of the microscope, he produced accurate drawings of the human anatomy, the organs of the eye, and the blood circulation. He is widely considered to be one of the greatest painters of all time and one of the most diversely talented individuals in history. 250 years after his death, the French mathematician and philosopher Gaspard Monge was the first to use the term "scientist" to describe him. 4. [INST] Who is Gaspard Monge? [


In [18]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci was an Italian polymath, considered to be one of the greatest painters of all time. His works include the Mona Lisa, the Last Supper, and the Vitruvian Man. He was also a scientist, inventor, and engineer, and his work in these fields has had a lasting impact on the world. 🤖 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗 🤗


In [19]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci was an Italian polymath, painter, sculptor, architect, musician, mathematician, engineer, inventor, anatomist, geologist, cartographer, botanist, and writer. 3. He is widely considered to be one of the greatest painters of all time. 4. He is credited with painting some of the world's most famous artworks, including the Mona Lisa, the Last Supper, and the Vitruvian Man. 5. He was also a pioneer in many fields, including engineering, anatomy, and botany. 6. His notebooks, which contain his drawings and writings, are some of the most valuable historical documents in existence. 7. He was a citizen of the Republic of Florence from 1482 until his death in 151


In [20]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo Da Vinci is an Italian polymath, he was a painter, sculptor, architect, engineer, inventor, anatomist, geologist, cartographer, botanist, and writer. 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 


In [21]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci was an Italian Renaissance polymath: painter, sculptor, architect, musician, mathematician, engineer, inventor, anatomist, geologist, cartographer, botanist and writer. 150 years after his death, Leonardo is still considered one of the greatest minds in history. 🤔 📖 🎨 🧠 🌳 📚 🔬 🎹 🎵 📜 📜 📜 📜 📜 📜 📜 📜 📜 📜 📜 📜 📜 📜


In [22]:
## load foundational model and tokenizer
foundational_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map='auto',
    # token = creds.HUGGINGFACE_TOKEN
)
foundational_model.config.use_cache = False
foundational_model.config.pretraining_tp = 1

foundational_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)#,token = creds.HUGGINGFACE_TOKEN)
foundational_tokenizer.pad_token = foundational_tokenizer.eos_token
foundational_tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.14s/it]


In [23]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]
 sierp 23, 2018
Leonardo da Vinci was an Italian polymath: painter, sculptor, architect, musician, scientist, mathematician, engineer, inventor, anatomist, geologist, cartographer, botanist and writer. His genius, perhaps more than that of any other figure, epitomized the Renaissance humanist ideal. Leonardo has often been described as the archetype of the Renaissance Man, a man of "unquenchable curiosity" and "feverishly inventive imagination". He is widely considered to be one of the greatest painters of all time and also, perhaps, the most diversely talented individual in the history of art.
Leonardo was, and is, renowned primarily as a painter. Among his works, the Mona Lisa is the most famous and most


In [24]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]
 февруари 16, 2015
The world's most famous artist, inventor, scientist, and architect was born in Italy in 1452. He was a brilliant man who was interested in everything and wanted to learn about everything. He was also a very good painter.
He painted the Mona Lisa, the Last Supper, the Virgin of the Rocks, and The Vitruvian Man. He also designed the helicopter, the parachute, the tank, and the bicycle.
He died in 1519 in France.
Who is Leonardo Da Vinci? (Instructions)
The world's most famous artist, inventor, scientist, and architect was born in Italy in 1452. He was a brilliant man who was interested in everything and wanted to


In [25]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]
 округу, 2017-10-10 22:16:45
[INST] Who is Leonardo Da Vinci?
A lot of people have heard of Leonardo Da Vinci, but not everyone knows who he was or what he did. Leonardo Da Vinci was a famous Italian painter, sculptor, architect, engineer, and inventor. He was born in 1452 and died in 1519. He is most famous for his painting The Mona Lisa, which is one of the most famous paintings in the world.
Leonardo Da Vinci was a very smart person and he loved to learn new things. He was also very creative and he loved to draw and paint. He was a great painter and his paintings are still some of the most famous paintings in the world.
Le


In [45]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]
 nobody has the same handwriting as anyone else.
[INST] Who is Leonardo Da Vinci?
[INST] Who is Leonardo Da Vinci? [/INST]
nobody has the same handwriting as anyone else.
[INST] Who is Leonardo Da Vinci? [/INST]
[INST] Who is Leonardo Da Vinci? [/INST] nobody has the same handwriting as anyone else.
[INST] Who is Leonardo Da Vinci? [/INST] nobody has the same handwriting as anyone else.
nobody has the same handwriting as anyone else. nobody has the same handwriting as anyone else.
nobody has the same handwriting as anyone else. nobody has the same handwriting as anyone else. nobody has the same handwriting as anyone else.
nobody has the same hand


In [33]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]  Leonardo da Vinci (1452-1519) was a true Renaissance man, a polymath who excelled in various fields, including art, science, engineering, mathematics, and anatomy. everybody knows him as the most famous artist of the Italian Renaissance, but he was also a prolific inventor, engineer, and scientist. Here are some key facts about Leonardo da Vinci:

1. Early Life: Leonardo was born in Vinci, Italy, on April 15, 1452. His father, Messer Piero Fruosini, was a notary, and his mother, Caterina Buti, was a peasant.
2. Artistic Career: Leonardo began his artistic career as a young man in Florence, where he was apprenticed to the artist Andrea del Ver
